In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
df =  pd.read_parquet('logs.parquet')
df.info()

In [55]:
usuarios = df.groupby('Usuario')
userdf = pd.DataFrame()
usuarios.size()

Usuario
#0 - null                         10721
#1002 - MOBIHOGAR                   128
#1005 - JESUS BERLANGA HERMOSO       95
#1009 - ANTONIO BREY SILVA           46
#1010 - Muebles Ibárrez              31
                                  ...  
#9313 - MAGRANS LOPEZ OSCAR          19
#936 - MUEBLES MESA                   9
#9441 - PEDIDOS MUEBLES CARLOS       76
#97 - COMODAS Y CAMAS                57
#980 - RENDON E HIJOS S.L.            4
Length: 769, dtype: int64

In [ ]:
userdf['TotalEventos'] = usuarios.size()
event_counts_per_user = df.groupby('Usuario')['Evento'].value_counts().unstack(fill_value=0)

event_perc_per_user = event_counts_per_user.apply(
        lambda x: x / x.sum() * 100 if x.sum() > 0 else 0, axis=1
        )
event_perc_per_user = event_perc_per_user.add_prefix('Perc_')
event_perc_per_user

userdf = userdf.join(event_perc_per_user)


In [57]:
userdf['DuracionMedia'] = usuarios['minutos'].mean()
userdf['DiseñosUnicos'] = usuarios.apply(
        lambda x: x.loc[x['diseñoid'].notna() & (x['diseñoid'] > 0), 'diseñoid'].nunique(),
        include_groups=False
    )
userdf['PiezasUnicas'] = usuarios.apply(
        lambda x: x.loc[x['Evento'] == 'AÑADIR_PIEZA', 'pieza'].nunique(),
        include_groups=False
    )
userdf['CatalogosUnicos'] = usuarios.apply(
        lambda x: x.loc[x['Evento'] == 'AÑADIR_PIEZA', 'catalogo'].nunique(),
        include_groups=False
    )

userdf.fillna(0, inplace=True) 

In [66]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(userdf)
n_clusters = 6  # Define el número de clusters deseados
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
userdf["Cluster"] = kmeans.fit_predict(features_scaled)
userdf.sort_values("Cluster")

,TotalEventos,Perc_ACEPTAR_PRIVACIDAD,Perc_AÑADIR_PIEZA,Perc_BORRAR_DISEÑO,Perc_CAMBIO_CONTRASEÑA,Perc_CARGAR_DISENO,Perc_CONEXION,Perc_CREAR_DISENO,Perc_DESCONEXION,Perc_DURACION,...,Perc_LOGOUT,Perc_NUEVO_LOGO,Perc_SOLICITUD_RESET,Perc_VER_PEDIDO,Perc_VER_PRESUPUESTO,DuracionMedia,DiseñosUnicos,PiezasUnicas,CatalogosUnicos,Cluster
Usuario,,,,,,,,,,,,,,,,,,,,,
#8762 - GALAN MARTIN MARIA ISABEL,36,0,23,0,0,0,0,1,3,3,...,3,0,0,0,0,102.975838,1,20,2,0
#867 - MUEBLES MARIO,79,0,40,0,0,4,0,2,6,6,...,6,0,0,0,7,81.916937,2,27,2,0
#8621 - ZASMOBEL . MUEBLES RAMIREZ S.L.,84,0,40,0,0,3,0,5,9,9,...,9,0,0,0,0,226.265599,6,37,6,0
#3599 - COMPTACREA,17,0,2,0,0,1,0,1,2,2,...,2,0,0,0,1,63.580042,1,2,1,0
#838 - MUEBLES MARTIN,27,0,5,0,0,2,0,1,4,4,...,4,0,0,0,0,58.115924,2,5,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#13934 - MUEBLES EL PARAISO. S.A.,321,0,162,0,0,33,0,9,26,26,...,26,0,0,0,8,91.456646,23,118,7,4
#1245 - MUEBLES JOVI,295,0,133,1,0,33,0,11,21,21,...,21,0,0,1,18,281.423903,14,94,10,4
#13380 - nuit et sommeil,379,0,143,0,0,51,0,5,31,31,...,31,0,0,0,35,86.485365,6,40,2,4
